In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col
from pyspark.sql.types import IntegerType,FloatType
spark=SparkSession.builder.appName('UDF').getOrCreate()
df=spark.read.options(header=True, inferSchema=True).csv('Files/OfficeData.csv')
df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 22:26:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/12 22:26:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/12 22:26:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/12 22:26:43 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/12 22:26:43 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/11/12 22:26:43 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [2]:
def get_total_salary(salary,bonus):
    return salary+bonus

totalSalary=udf(lambda x,y: get_total_salary(x,y),IntegerType())

df.withColumn('Total Salary',totalSalary(df.salary,col('bonus'))).show()
    

+-------------+----------+-----+------+---+-----+------------+
|employee_name|department|state|salary|age|bonus|Total Salary|
+-------------+----------+-----+------+---+-----+------------+
|        James|     Sales|   NY| 90000| 34|10000|      100000|
|      Michael|     Sales|   NY| 86000| 56|20000|      106000|
|       Robert|     Sales|   CA| 81000| 30|23000|      104000|
|        Maria|   Finance|   CA| 90000| 24|23000|      113000|
|        Raman|   Finance|   CA| 99000| 40|24000|      123000|
|        Scott|   Finance|   NY| 83000| 36|19000|      102000|
|          Jen|   Finance|   NY| 79000| 53|15000|       94000|
|         Jeff| Marketing|   CA| 80000| 25|18000|       98000|
|        Kumar| Marketing|   NY| 91000| 50|21000|      112000|
+-------------+----------+-----+------+---+-----+------------+



In [3]:
def hikedSalary(salary):
    return salary*0.05+salary

totalhikedSalary=udf(lambda x: hikedSalary(x), FloatType())

df.withColumn("Hiked Salary",totalhikedSalary(df.salary)).show()

+-------------+----------+-----+------+---+-----+------------+
|employee_name|department|state|salary|age|bonus|Hiked Salary|
+-------------+----------+-----+------+---+-----+------------+
|        James|     Sales|   NY| 90000| 34|10000|     94500.0|
|      Michael|     Sales|   NY| 86000| 56|20000|     90300.0|
|       Robert|     Sales|   CA| 81000| 30|23000|     85050.0|
|        Maria|   Finance|   CA| 90000| 24|23000|     94500.0|
|        Raman|   Finance|   CA| 99000| 40|24000|    103950.0|
|        Scott|   Finance|   NY| 83000| 36|19000|     87150.0|
|          Jen|   Finance|   NY| 79000| 53|15000|     82950.0|
|         Jeff| Marketing|   CA| 80000| 25|18000|     84000.0|
|        Kumar| Marketing|   NY| 91000| 50|21000|     95550.0|
+-------------+----------+-----+------+---+-----+------------+



In [14]:
def incremented(salary,state,bonus):
    if state=='NY':
        return salary*.1+bonus*.05
    elif state=='CA':
        return salary*.12+bonus*.03
    
increment=udf(lambda x,y,z : incremented(x,y,z),FloatType())

df.withColumn('increment',increment(df.salary,df.state,df.bonus)).show()

+-------------+----------+-----+------+---+-----+---------+
|employee_name|department|state|salary|age|bonus|increment|
+-------------+----------+-----+------+---+-----+---------+
|        James|     Sales|   NY| 90000| 34|10000|   9500.0|
|      Michael|     Sales|   NY| 86000| 56|20000|   9600.0|
|       Robert|     Sales|   CA| 81000| 30|23000|  10410.0|
|        Maria|   Finance|   CA| 90000| 24|23000|  11490.0|
|        Raman|   Finance|   CA| 99000| 40|24000|  12600.0|
|        Scott|   Finance|   NY| 83000| 36|19000|   9250.0|
|          Jen|   Finance|   NY| 79000| 53|15000|   8650.0|
|         Jeff| Marketing|   CA| 80000| 25|18000|  10140.0|
|        Kumar| Marketing|   NY| 91000| 50|21000|  10150.0|
+-------------+----------+-----+------+---+-----+---------+



In [15]:
type(df)

pyspark.sql.dataframe.DataFrame

In [16]:
rdd=df.rdd
type(rdd)

pyspark.rdd.RDD